<a href="https://colab.research.google.com/github/Albayt54/uas_datamining/blob/main/YoutubeCommentScrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from googleapiclient.discovery import build
from collections import Counter
import streamlit as st
from Senti import extract_video_id
from googleapiclient.errors import HttpError

csv: Modul ini menyediakan fungsi untuk membaca dan menulis file CSV (Comma Separated Values).

googleapiclient.discovery.build: Fungsi ini digunakan untuk membuat objek layanan untuk berinteraksi dengan Google API. Ini adalah bagian dari Google API Client Library untuk Python.

collections.Counter: Ini adalah subkelas dari dictionary yang disediakan oleh modul collections, yang membantu menghitung objek yang bisa di-hash. Ini biasanya digunakan untuk menghitung elemen dalam iterable.

streamlit as st: Ini mengimpor library Streamlit, yang digunakan untuk membuat aplikasi web untuk proyek data sains dan machine learning dengan sedikit usaha.

from Senti import extract_video_id: Ini mengimpor fungsi bernama extract_video_id dari modul bernama Senti. Fungsi ini kemungkinan mengekstrak ID video dari input yang diberikan.

from googleapiclient.errors import HttpError: Ini mengimpor kelas HttpError, yang digunakan untuk menangani kesalahan HTTP yang terjadi saat berinteraksi dengan Google API.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

**warnings.filterwarnings('ignore'): Mengabaikan semua peringatan yang mungkin muncul selama eksekusi kode.**

In [ ]:
# Replace with your own API key
DEVELOPER_KEY = st.secrets["API_KEY"]
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'
# Create a client object to interact with the YouTube API
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

**st.secrets["API_KEY"]: Mengambil kunci API dari secrets Streamlit untuk mengakses YouTube API.**

**build(): Fungsi untuk membuat objek klien yang akan digunakan untuk berinteraksi dengan YouTube API.**

video_id=extract_video_id(youtube_link)

In [ ]:
def get_channel_id(video_id):
    response = youtube.videos().list(part='snippet', id=video_id).execute()
    channel_id = response['items'][0]['snippet']['channelId']
    return channel_id

**youtube.videos().list(part='snippet', id=video_id).execute(): Meminta informasi video berdasarkan ID video.
channel_id: Mendapatkan ID channel dari respons yang diterima.**

channel_id=get_channel_id(video_id)

In [ ]:
def save_video_comments_to_csv(video_id):
    # Retrieve comments for the specified video using the comments().list() method
    comments = []
    results = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        textFormat='plainText'
    ).execute()

    # Extract the text content of each comment and add it to the comments list
    while results:
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            username = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
            comments.append([username,comment])
        if 'nextPageToken' in results:
            nextPage = results['nextPageToken']
            results = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                textFormat='plainText',
                pageToken=nextPage
            ).execute()
        else:
            break

    # Save the comments to a CSV file with the video ID as the filename
    filename = video_id + '.csv'
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Username','Comment'])
        for comment in comments:
            writer.writerow([comment[0],comment[1]])

    return filename

**youtube.commentThreads().list(): Meminta komentar untuk video yang ditentukan.
comments: Daftar yang digunakan untuk menyimpan komentar dan nama pengguna.**

**while results: Loop untuk mengumpulkan semua komentar, termasuk halaman berikutnya jika ada.**

**csv.writer: Menulis komentar yang diambil ke file CSV.**

In [ ]:
def get_video_stats(video_id):
    try:
        response = youtube.videos().list(
            part='statistics',
            id=video_id
        ).execute()

        return response['items'][0]['statistics']

    except HttpError as error:
        print(f'An error occurred: {error}')
        return None

**youtube.videos().list(part='statistics', id=video_id).execute(): Meminta statistik video berdasarkan ID video.**

**try-except: Menangani kesalahan HTTP yang mungkin terjadi selama permintaan API.**

In [ ]:
def get_channel_info(youtube, channel_id):
    try:
        response = youtube.channels().list(
            part='snippet,statistics,brandingSettings',
            id=channel_id
        ).execute()

        channel_title = response['items'][0]['snippet']['title']
        video_count = response['items'][0]['statistics']['videoCount']
        channel_logo_url = response['items'][0]['snippet']['thumbnails']['high']['url']
        channel_created_date = response['items'][0]['snippet']['publishedAt']
        subscriber_count = response['items'][0]['statistics']['subscriberCount']
        channel_description = response['items'][0]['snippet']['description']


        channel_info = {
            'channel_title': channel_title,
            'video_count': video_count,
            'channel_logo_url': channel_logo_url,
            'channel_created_date': channel_created_date,
            'subscriber_count': subscriber_count,
            'channel_description': channel_description
        }

        return channel_info

    except HttpError as error:
        print(f'An error occurred: {error}')
        return None

**youtube.channels().list(part='snippet,statistics,brandingSettings', id=channel_id).execute(): Meminta informasi channel berdasarkan ID channel.**

**channel_info: Mengumpulkan informasi tentang channel seperti judul, jumlah video, URL logo, tanggal pembuatan, jumlah pelanggan, dan deskripsi.**

**try-except: Menangani kesalahan HTTP yang mungkin terjadi selama permintaan API.**